We want to group by multi-valued columns. For example, take this dataset:

In [1]:
import pandas as pd
data = pd.DataFrame([
        [['a', 'b'], ['x', 'y'], 3],
        [['a', 'c'], ['x', 'z'], 2],
    ], columns=['left', 'right', 'value'])
data

,left,right,value
0,"[a, b]","[x, y]",3
1,"[a, c]","[x, z]",2


`left` and `right` have arrays as values.

We want the equivalent of a `.groupby([left, right])['value'].sum()` that would return:

    left right value
       a     x     5    # (a, x) occurs in the first AND second row
       a     y     3    # from first row
       b     x     3    # from first row
       b     y     3    # from first row
       a     z     2    # from second row
       c     x     2    # from second row
       c     z     2    # from second row

### Approach 1: Python

Let's write this as a series of `for` loops first:

In [2]:
def python_kpartite(data, left, right, value):
    result = {}
    for index, row in data.iterrows():            # Loop through every row
        for a in row[left]:                       # Loop through all values in the left cell
            for b in row[right]:                  # ... and the right cell
                if (a, b) in result:              # Add value to the results counter
                    result[a, b] += row[value]
                else:
                    result[a, b] = row[value]
    return result                                 # Return the dict

python_kpartite(data, 'left', 'right', 'value')

{('a', 'x'): 5L,
 ('a', 'y'): 3L,
 ('a', 'z'): 2L,
 ('b', 'x'): 3L,
 ('b', 'y'): 3L,
 ('c', 'x'): 2L,
 ('c', 'z'): 2L}

This result is correct. But it is fairly slow even for small data.

In [5]:
smalldata = pd.concat([data] * 10000)
middata = pd.concat([data] * 100000)

In [6]:
%timeit python_kpartite(smalldata, 'left', 'right', 'value')

1 loops, best of 3: 2.46 s per loop


### Python with Pandas

In [7]:
result = {}
def python_inner(row):
    for a in row['left']:
        for b in row['right']:
            if (a, b) in result:
                result[a, b] += row['value']
            else:
                result[a, b] = row['value']

In [8]:
%timeit smalldata.apply(python_inner, axis=1)

1 loops, best of 3: 1.44 s per loop


### Approach 2: Cython

In [9]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


#### Naive Cython is not faster

In [10]:
%%cython

# This is a one-time compilation! If you re-run it, result will still be cached.
# Change this comment to recompile.
result = {}
def cython_inner(row):
    for a in row['left']:
        for b in row['right']:
            result[a, b] = result.get((a, b), 0) + row['value']

In [11]:
%timeit smalldata.apply(cython_inner, axis=1)

1 loops, best of 3: 1.42 s per loop


### Approach 3: Cython on NumPy

See http://nbviewer.ipython.org/gist/tillahoffmann/296501acea231cbdf5e7 for profiling

In [10]:
#Load Robert Kern's line profiler
%load_ext line_profiler
import line_profiler

In [11]:
#Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
from Cython.Compiler.Options import directive_defaults

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [12]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1

cimport numpy as np

def cython_kpartite(np.ndarray npdata):
    cdef int index
    cdef np.ndarray row
    cdef list left, right
    cdef double value
    cdef str a, b
    cdef tuple key
    cdef dict result = {}
    for index in range(npdata.shape[0]):
        row = npdata[index]
        left = row[0]
        right = row[1]
        value = row[2]
        for a in left:
            for b in right:
                key = a, b
                result[key] = result.get(key, 0)  + value
    return result

In [13]:
%timeit cython_kpartite(middata.values)

1 loops, best of 3: 294 ms per loop


### Optimisation

In [14]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1

cimport numpy as np

# cimport cython
# @cython.boundscheck(False)
# @cython.wraparound(False)
# @cython.overflowcheck(False)
# @cython.nonecheck(False)
def cython_kpartite2(                           # Split parameters on input (-80ms)
        np.ndarray[list] leftarray,
        np.ndarray[list] rightarray,
        np.ndarray[double] valuearray):
    cdef int index, n = leftarray.shape[0]
    # cdef list left, right
    # cdef double value
    result = {}
    for index in range(n):
        right = rightarray[index]
        value = valuearray[index]
        for a in leftarray[index]:
            for b in right:
                key = a, b                     # TODO: Use a string instead of a tuple (saves -80ms)
                result[key] = result.get(key, 0.0) + value
    return result

In [15]:
%timeit cython_kpartite2(middata.left.values, middata.right.values, middata.value.astype(float).values)

1 loops, best of 3: 219 ms per loop


In [16]:
profile = line_profiler.LineProfiler(cython_kpartite2)
profile.runcall(cython_kpartite2, middata.left.values, middata.right.values, middata.value.astype(float).values)
profile.print_stats()

Timer unit: 4.66511e-07 s

Total time: 1.90599 s
File: D:\cygwin64\home\Anand\.ipython\cython\_cython_magic_16adb21d1da83629b403b2790d247d8a.pyx
Function: cython_kpartite2 at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def cython_kpartite2(                           # Split parameters on input (-80ms)
    10                                                   np.ndarray[list] leftarray,
    11                                                   np.ndarray[list] rightarray,
    12                                                   np.ndarray[double] valuearray):
    13         1            4      4.0      0.0      cdef int index, n = leftarray.shape[0]
    14                                               # cdef list left, right
    15                                               # cdef double value
    16         1            1      1.0      0.0      result = {}
    17         1            1      1.0      0.0      f

In [33]:
%%cython

cimport numpy as np

def bipartite(np.ndarray[list] leftarray, np.ndarray[list] rightarray):
    cdef int index, n = leftarray.shape[0]
    cdef dict result = {}
    cdef list right
    cdef str a, b
    for index in range(n):
        right = rightarray[index]
        for a in leftarray[index]:
            for b in right:
                key = a, b
                result[key] = result.get(key, 0) + 1
    return result

In [34]:
%timeit bipartite(middata.left.values, middata.right.values)

1 loops, best of 3: 196 ms per loop


----

# Generalised K-Partite

Let's generalise the problem. If we want multi-partite counts across multiple columns, and each could be multi-valued, like this:

    ColA    ColB    ColC
    A1,A2   B1      C1,C2,C3
    A1,A3   -       C1
    A1      B1,B2   C2,C3
    
In that case, the link pairs we want are:

    ColA    ColB    ColC       =>    Internal links               External links
    A1,A2   B1      C1,C2,C3         A1-A2, C1-C2, C1-C3, C2-C3   A1-B1, A1-C1, A1-C2, A1-C3, A2-B1, A2-C1, A2-C2, A2-C3
    A1,A3   -       C1               A1-A3                        A1-C1, A3-C1
    A1      B1,B2   C2,C3            B1-B2, C2-C3                 A1-B1, A1-B2, A1-C2, A1-C3, B1-C2, B1-C3, B2-C2, B2-C3

The psuedo-code for this is:

    for row in data:                                              # Handle every row independently
        for column1 in columns:                                   # Loop through every column
            for value1 in row[column1]:                           # Loop through each item in the list
                for column2 in columns:                           # Loop through every column again
                    if column2 < column1:                         # But ignore columns already examined
                        continue
                   for value2 in row[column2]:                    # Loop through each item in the list
                       count[value1, value2] += 1                 # Increment the counter for every pair of values
                               
Let's get this right in Python first.

In [93]:
import pandas as pd
from collections import Counter

In [94]:
data = pd.DataFrame({
        'ColA': [['A1', 'A2'], ['A1','A3'], ['A1']],
        'ColB': [['B1'], [], ['B1', 'B2']],
        'ColC': [['C1', 'C2', 'C3'], ['C1'], ['C2', 'C3']]
    })

In [95]:
def python_kpartite(data):
    columns = range(len(data.columns))
    count = Counter()
    for row in data.values:
        for column1 in columns:
            for value1 in row[column1]:
                for column2 in columns:
                    if column2 < column1:
                        continue
                    for value2 in row[column2]:
                        count[value1, value2] += 1
    return pd.Series(count)

## How fast is it?

Do we even need to optimise it? Let's take a look at it's speed on 30,000 rows:

In [4]:
%timeit python_kpartite(pd.concat([data]*10000))

1 loops, best of 3: 1.21 s per loop


Let's look at some real-life data

In [5]:
airline = pd.read_csv('airline-patents.csv')

# Convert all string columns to arrays -- split by ";"
columns = [col for col in airline.columns if airline[col].dtype.kind == 'O']
for col in columns:
    airline[col] = airline[col].str.split(';')

In [6]:
airline[columns].head()

,PN,KD,AN,CT,FCT,keywords,IPC,PA,INV
0,[US2006244310],[AA],[US20050165942],"[""]","[""]",[brake pedal],[B60T],[BOEING CO],[DEVLIEG GARRETT H]
1,[US2006244310],[AA],[US20050165942],"[""]","[""]",[brake pedal],[B64C],[BOEING CO],[DEVLIEG GARRETT H]
2,[US2010256884],[AA],[US20070625681],"[""]","[""]",[vehicle speed],[B64C],[BOEING CO],[DEVLIEG GARRETT HOWARD]
3,[US2010256884],[AA],[US20070625681],"[""]","[""]",[vehicle speed],[B60T],[BOEING CO],[DEVLIEG GARRETT HOWARD]
4,[US2010256884],[AA],[US20070625681],"[""]","[""]",[vehicle speed],[G06F],[BOEING CO],[DEVLIEG GARRETT HOWARD]


In [7]:
len(airline)

210090

In [8]:
%timeit python_kpartite(airline[columns].head(1000))

1 loops, best of 3: 3.86 s per loop


So, to process 1,000 rows out of 210,090 (~1/200th of the data) it takes about 4 seconds.

We'd like this to be at least 800 times faster -- process the entire dataset in one second.

## Naive Cython

Let's just copy the same algorithm into Cython and see how fast it is.

In [9]:
%load_ext Cython

In [59]:
%%cython

from collections import Counter

def cython_kpartite(data):
    columns = range(len(data.columns))
    count = Counter()
    for row in data.values:
        for column1 in columns:
            for value1 in row[column1]:
                for column2 in columns:
                    if column2 < column1:
                        continue
                    for value2 in row[column2]:
                        count[value1, value2] += 1
    return count

In [60]:
%timeit cython_kpartite(airline[columns].head(1000))

1 loops, best of 3: 3.07 s per loop


### It's Counter that's slow

Actually, let's run *the same algorithm* but instead of incrementing a `collections.Counter`, lets increment an integer and see how much time it takes.

In [61]:
%%cython

def cython_kpartite(data):
    columns = range(len(data.columns))
    count = 0
    for row in data.values:
        for column1 in columns:
            for value1 in row[column1]:
                for column2 in columns:
                    if column2 < column1:
                        continue
                    for value2 in row[column2]:
                        count += 1
    return count

In [62]:
%timeit cython_kpartite(airline[columns].head(1000))

10 loops, best of 3: 121 ms per loop


That's about 25 times as fast. Not bad, but not good enough. Plus, we need to see how to optimise `collections.Counter`.

### We're creating too many links!

Let's see how many links this will create:

In [63]:
print '{:,d}'.format(cython_kpartite(airline[columns].head(1000)))
print '{:,d}'.format(cython_kpartite(airline[columns].head(20000)))

5,180,892
41,612,299


That's **5 MILLION** links for just the first 1,000 rows, and **41 MILLION** links for 20,000 rows, i.e. 10% of the data.

Obviously, we're not interested in the long tail. We'd be primarily interested in the connections between the largest nodes. So we need to rework our approach:

1. Find the N largest nodes (where N can be specified by the user)
2. Find the links between the largest nodes

### Find the N largest nodes

Since every value is associated with every other value in a row, calculating this is simple:

In [177]:
%%cython

from collections import defaultdict

def cython_count(data, top=0):
    cdef int ncols = len(data.columns)              # ncols = number of columns in data
    cdef int rowsize                                # rowsize = number of nodes in each row
    count = defaultdict(int)                        # count[node] = number of links the node has
    for row in data.values:                         # We loop through each row
        rowsize = 0
        for col in range(ncols):                    # ... and calculate the number of nodes in this row
            rowsize += len(row[col])
        for col in range(ncols):                    # In every column...
            values = row[col]
            for index in range(len(values)):        # ... for every node...
                count[col, values[index]] += rowsize     # ... we add the number of nodes in the row to its count
    
    # Get the top values. This is quite fast
    if top > 0:
        count = dict(sorted(count.items(), key=lambda v: -v[1])[:top])

    return count                                    # Count is now a dict that has the number of links against each node

In [178]:
%timeit cython_count(airline[columns].head(100000), 1000)

1 loops, best of 3: 1.22 s per loop


That's reasonably fast (but incidentally, even here, the bulk of the time goes into `defaultdict(int)`. We need a faster counter.)

## Link the top N nodes

Let's just create the links for the top nodes. We can skip anything that's not in that list.

Here's what we could do in every row:

1. Get the (column, value) for every column and value
2. Map it to a node index
3. Increment all combinations of every node index pairs

In [212]:
%%cython

from collections import defaultdict

def cython_kpartite(data, top=10000):
    cdef int ncols = len(data.columns)              # ncols = number of columns in data
    cdef int rowsize                                # rowsize = number of nodes in each row
    count = defaultdict(int)                        # count[node] = number of links the node has
    for row in data.values:                         # We loop through each row
        rowsize = 0
        for col in range(ncols):                    # ... and calculate the number of nodes in this row
            rowsize += len(row[col])
        for col in range(ncols):                    # In every column...
            values = row[col]
            for index in range(len(values)):        # ... for every node...
                count[col, values[index]] += rowsize     # ... we add the number of nodes in the row to its count
    
    # Get the top values. This is quite fast
    if top > 0:
        count = dict(sorted(count.items(), key=lambda v: -v[1])[:top])

    # For the top nodes, let's get the (column, value) -> node-index
    nodes = count.keys()
    index = {key: i for i, key in enumerate(count)}
    
    # Here's a slow implementation of the logic
    links = defaultdict(int)
    for row in data.values:
        indices = []
        for col in range(ncols):
            values = row[col]
            for i in range(len(values)):
                key = (col, values[i])
                if key in index:
                    indices.append(index[key])
        n = len(indices)
        for i in range(n):
            for j in range(i + 1, n):
                links[i, j] += 1
    return nodes, links

In [214]:
cython_kpartite(airline[columns].head(1000), 10)

([(3, 'US4489123'),
  (7, 'BOEING CO'),
  (3, 'US4935291'),
  (4, 'US7194391BB'),
  (3, 'US5160771'),
  (3, 'US4992323'),
  (1, 'A'),
  (3, 'US4500589'),
  (3, 'US5429326'),
  (4, 'US7753312BB')],
 defaultdict(<type 'int'>, {(5, 9): 353, (4, 7): 353, (1, 3): 365, (4, 8): 353, (5, 6): 365, (2, 8): 353, (6, 9): 353, (0, 7): 353, (8, 9): 353, (1, 6): 365, (3, 7): 353, (2, 5): 365, (0, 3): 365, (5, 8): 353, (1, 2): 401, (4, 9): 353, (2, 9): 353, (1, 5): 365, (3, 6): 365, (0, 4): 365, (7, 9): 353, (2, 7): 353, (2, 6): 365, (4, 5): 365, (1, 4): 365, (3, 9): 353, (0, 5): 365, (1, 9): 353, (6, 8): 353, (6, 7): 353, (0, 8): 353, (3, 5): 365, (0, 1): 951, (4, 6): 365, (7, 8): 353, (5, 7): 353, (0, 2): 401, (3, 8): 353, (0, 6): 365, (1, 8): 353, (1, 7): 353, (0, 9): 353, (2, 3): 365, (3, 4): 365, (2, 4): 365}))

Functionally this works fine. Now let's optimise it.